In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
import warnings
warnings.filterwarnings("ignore")

import seaborn as sb
from sklearn import model_selection
from sklearn.model_selection import KFold

# **APPLE (2016-present)**

In [ ]:
Apple=pd.read_csv("/kaggle/input/d/advikmaniar/popular-stock-prices/AppleStocks.csv")
display(Apple)

In [ ]:
Apple["Date"]=pd.to_datetime(Apple.Date,format="%Y-%m-%d")
Apple.index=Apple['Date']

plt.figure(figsize=(20,14))
plt.subplot(2,1,1)
plt.plot(Apple["Open"],label='Open Price history')
plt.plot(Apple["High"],label='High Price history')
plt.plot(Apple["Low"],label='Low Price history')
plt.plot(Apple["Close"],label='Close Price history')
plt.plot(Apple["Adj Close"],label='Adj Close Price history')
plt.grid()
plt.legend()
plt.show()

In [ ]:
#Sort the dataset on date time and filter “Date” and “Close” columns
Apple1=Apple.sort_index(ascending=True,axis=0)
new_dataset=pd.DataFrame(index=range(0,len(Apple)),columns=['Date','Close'])
L=len(Apple)

for i in range(0,len(Apple1)):
    new_dataset["Date"][i]=Apple1['Date'][i]
    new_dataset["Close"][i]=Apple1["Close"][i]

#Normalize the Dataset
scaler=MinMaxScaler(feature_range=(0,1))
new_dataset.index=new_dataset.Date
new_dataset.drop("Date",axis=1,inplace=True)
final_dataset=new_dataset.values

train_data=final_dataset[0:L-300,:]
valid_data=final_dataset[L-300:,:]

scaled_data=scaler.fit_transform(final_dataset)

x_train_data,y_train_data=[],[]

for i in range(300,len(train_data)):
    x_train_data.append(scaled_data[i-300:i,0])
    y_train_data.append(scaled_data[i,0])

In [ ]:
x_train_data,y_train_data=np.array(x_train_data),np.array(y_train_data)
x_train_data=np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.shape[1],1))

In [ ]:
#Build and train the LSTM model
lstm_model=Sequential()
lstm_model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train_data.shape[1],1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

inputs_data=new_dataset[len(new_dataset)-len(valid_data)-300:].values
inputs_data=inputs_data.reshape(-1,1)
inputs_data=scaler.transform(inputs_data)

lstm_model.compile(loss='mean_squared_error',optimizer='adam')
lstm_model.fit(x_train_data,y_train_data,epochs=1,batch_size=1,verbose=2)

In [ ]:
#Testing data for the model
X_test=[]
for i in range(300,inputs_data.shape[0]):
    X_test.append(inputs_data[i-300:i,0])
    
X_test=np.array(X_test)
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

predicted_closing_price=lstm_model.predict(X_test)
predicted_closing_price=scaler.inverse_transform(predicted_closing_price)

In [ ]:
train_data=new_dataset[:L-300]
valid_data=new_dataset[L-300:]
valid_data['Predictions']=predicted_closing_price
display(valid_data)

#Plot predicted stock values
plt.figure(figsize=(20,8))
plt.plot(train_data["Close"])
plt.plot(valid_data['Close'],label = "Real Value")
plt.plot(valid_data["Predictions"],label = "Prediction")

plt.legend()
plt.grid()
plt.show()

# **TATA 2016-present**

In [ ]:
Tata=pd.read_csv("/kaggle/input/d/advikmaniar/popular-stock-prices/TataStocks.csv")
Tata

In [ ]:
Tata["Date"]=pd.to_datetime(Tata.Date,format="%Y-%m-%d")
Tata.index=Tata['Date']

plt.figure(figsize=(20,8))
plt.plot(Tata["Open"],label='Open Price history')
plt.plot(Tata["High"],label='High Price history')
plt.plot(Tata["Low"],label='Low Price history')
plt.plot(Tata["Close"],label='Close Price history')
plt.grid()
plt.legend()
plt.show()

In [ ]:
#Sort the dataset on date time and filter “Date” and “Close” columns
data1=Tata.sort_index(ascending=True,axis=0)
new_dataset=pd.DataFrame(index=range(0,len(Tata)),columns=['Date','Close'])
L=len(Tata)

for i in range(0,len(data1)):
    new_dataset["Date"][i]=data1['Date'][i]
    new_dataset["Close"][i]=data1["Close"][i]

    #Normalize the Dataset
scaler=MinMaxScaler(feature_range=(0,1))
new_dataset.index=new_dataset.Date
new_dataset.drop("Date",axis=1,inplace=True)
final_dataset=new_dataset.values

train_data=final_dataset[0:L-300,:]
valid_data=final_dataset[L-300:,:]

scaled_data=scaler.fit_transform(final_dataset)

x_train_data,y_train_data=[],[]

for i in range(300,len(train_data)):
    x_train_data.append(scaled_data[i-300:i,0])
    y_train_data.append(scaled_data[i,0])

In [ ]:
x_train_data,y_train_data=np.array(x_train_data),np.array(y_train_data)
x_train_data=np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.shape[1],1))

In [ ]:
#Build and train the LSTM model
lstm_model=Sequential()
lstm_model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train_data.shape[1],1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

inputs_data=new_dataset[len(new_dataset)-len(valid_data)-300:].values
inputs_data=inputs_data.reshape(-1,1)
inputs_data=scaler.transform(inputs_data)

lstm_model.compile(loss='mean_squared_error',optimizer='adam')
lstm_model.fit(x_train_data,y_train_data,epochs=1,batch_size=1,verbose=2)

In [ ]:
#Testing data for the model
X_test=[]
for i in range(300,inputs_data.shape[0]):
    X_test.append(inputs_data[i-300:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
predicted_closing_price=lstm_model.predict(X_test)
predicted_closing_price=scaler.inverse_transform(predicted_closing_price)

In [ ]:
train_data=new_dataset[:L-300]
valid_data=new_dataset[L-300:]
valid_data['Predictions']=predicted_closing_price

#Plot predictions
plt.figure(figsize=(20,8))
plt.plot(train_data["Close"])
plt.plot(valid_data['Close'],label = "Real Value")
plt.plot(valid_data["Predictions"],label = "Prediction")

plt.legend()
plt.grid()
plt.show()

# **Netflix (2016-present)**

In [ ]:
netflix=pd.read_csv("/kaggle/input/d/advikmaniar/popular-stock-prices/NFLX.csv")
netflix["Date"]=pd.to_datetime(netflix.Date,format="%Y-%m-%d")
netflix.index=netflix['Date']
netflix

In [ ]:
#Correlation of values
corr = netflix.corr(method='pearson')
corr

In [ ]:
#Dataset prepare
nflx_df=netflix[['Date','High','Open','Low','Close']]
display(nflx_df.head(10))

plt.figure(figsize=(20,8))
plt.title('Netflix Closing Price ')
plt.plot(nflx_df.index,nflx_df['Close'],label="Close")
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price',fontsize=18)
plt.legend()
plt.grid()
plt.show()

In [ ]:
#Specific values visualiation
nflx_df[['Open','Close']].head(20).plot(kind='bar')
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

nflx_df[['Low','Close']].head(20).plot(kind='bar')
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

nflx_df[['High','Close']].head(20).plot(kind='bar')
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
netflix['Date'] = pd.to_datetime(netflix.Date, format='%Y-%m-%d')
nflx_df['Year']=netflix['Date'].dt.year
nflx_df['Month']=netflix['Date'].dt.month
nflx_df['Day']=netflix['Date'].dt.day

nfx_df=nflx_df[['Day','Month','Year','High','Open','Low','Close']]
nfx_df.tail(10)

In [ ]:
#Separate Independent and dependent variable
X = nfx_df.iloc[:,nfx_df.columns !='Close']
Y= nfx_df.iloc[:, 5]

x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.3)
print(x_train.shape) 
print(x_test.shape) 
print(y_train.shape)
print(y_test.shape) 

In [ ]:
lr=LinearRegression()
lr.fit(x_train,y_train)

c=lr.intercept_
m=lr.coef_
print("Intercept is: {}".format(c))
print("Coefficients are: {}".format(m))

In [ ]:
y_pred=lr.predict(x_test)

#Check accuracy of model
kfold = model_selection.KFold(n_splits=20)
results_kfold = model_selection.cross_val_score(lr, x_test, y_test.astype('int'), cv=kfold)
print("Accuracy: ", results_kfold.mean()*100,"%")

# **Using Decision Tree**
__(For apple stocks)__

In [ ]:
future_days = 100
Apple['Prediction'] = Apple[['Close']].shift(-future_days)
Apple.tail()

In [ ]:
X = np.array(Apple.drop(["Date",'Prediction'], 1))[:-future_days]
print(X)

Y = np.array(Apple['Prediction'])[:-future_days]
print("Predictions: {}".format(Y))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.4)
tree = DecisionTreeRegressor().fit(x_train, y_train)
lr = LinearRegression().fit(x_train, y_train)

In [ ]:
x_future = Apple.drop(["Date",'Prediction'], 1)[:-future_days]
x_future = x_future.tail(future_days) 
x_future = np.array(x_future)

tree_prediction = tree.predict(x_future)
print( tree_prediction )
print()

lr_prediction = lr.predict(x_future)
print(lr_prediction)

In [ ]:
#Visualize the data
predictions = tree_prediction

#Plot the data
valid =  Apple[X.shape[0]:]
valid['Predictions'] = predictions #Create a new column called 'Predictions' that will hold the predicted prices
plt.figure(figsize=(20,8))
plt.xlabel('Days')
plt.ylabel('Close Price')
plt.plot(Apple['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Close', 'Val', 'Prediction' ], loc='lower right')
plt.grid()
plt.show()